1)  Напишите функцию без параметров, возвращающую генератор, который последовательно возвращает информацию о фильмах из Топ-500 КиноПоиска в виде словаря. 

2) Добавьте в функцию параметры, ограничивающие количество фильмов  и изменяющие время перерыва между запросами. Добавьте проверку на корректность переданных параметров функции - если передано некорректное значение аргумента, нужно выбрасывать подходящее исключение.



In [0]:
import urllib.request
import json
import time
import re
import datetime
from datetime import date
import urllib.request
from bs4 import BeautifulSoup
 
list_of_dicts = []

def find_films(num,sleep):
  if num > 500:
    raise ValueError('Maximum number of films - 500!')
  if type(num) is not int:
    raise TypeError('The number of films is needed!')
  if type(sleep) is str:
    raise TypeError('The number of seconds is needed')
  else:
    for i in range(5):
      url = 'https://www.kinoafisha.info/rating/movies/?page=' + str(i) 
      req = urllib.request.Request(url)
      response = urllib.request.urlopen(req)
      result = response.read().decode('utf-8')
      doc = BeautifulSoup(result, 'lxml')
      time.sleep(sleep)

      timestamp = datetime.datetime.now()
      _utc_timestamp = str(timestamp)[:19]

      divs = doc.findAll('div', {'class': 'films_right'})
      for div in divs:
        d = {}
        name_url = div.find_all('a', {'class': 'films_name ref'})
        for a in name_url:
          d['name'] = re.findall(r'>(.*?)</a>', str(a))[0]
          d['url'] = 'https://www.kinoafisha.info' + a.get('href')
          d['_utc_timestamp'] = _utc_timestamp
        span_rn = div.findAll('span', {'class': 'rating_num'})
        for span in span_rn:
          kp_rating = re.findall(r'>(.*?)</span>', str(span))
          d['kp_rating'] = kp_rating[0]

        span_fi = div.findAll('span', {'class': 'films_info'})
        for span in span_fi:
          span_n = re.findall(r'>(.*?)</span>', str(span))
          year_n = re.findall(r'\d{4},', str(span_n))
          if year_n != []:
            d['year'] = year_n[0][:-1]
        list_of_dicts.append(d)
        
    x=0
    for i in range(num):
      while x <= num-1:
        yield list_of_dicts[x]
        x += 1
      

#Например, напечатаем первые 5 фильмов (генератор 6й он не дает, т.к в параметрах указано только 5 фильмов)

generator_films = find_films(5,0.5)

print(next(generator_films))
print(next(generator_films))
print(next(generator_films))
print(next(generator_films))
print(next(generator_films))
print(next(generator_films))



{'name': 'FORD против FERRARI', 'url': 'https://www.kinoafisha.info/movies/8354457/', '_utc_timestamp': '2019-12-22 13:58:05', 'kp_rating': '9.4', 'year': '2019'}
{'name': 'Анастасия', 'url': 'https://www.kinoafisha.info/movies/5640854/', '_utc_timestamp': '2019-12-22 13:58:05', 'kp_rating': '9.2', 'year': '1997'}
{'name': 'Холодное сердце 2', 'url': 'https://www.kinoafisha.info/movies/8329619/', '_utc_timestamp': '2019-12-22 13:58:05', 'kp_rating': '9.1', 'year': '2019'}
{'name': 'Ржев', 'url': 'https://www.kinoafisha.info/movies/8356720/', '_utc_timestamp': '2019-12-22 13:58:05', 'kp_rating': '9.1', 'year': '2019'}
{'name': 'Малефисента: Владычица тьмы', 'url': 'https://www.kinoafisha.info/movies/8326540/', '_utc_timestamp': '2019-12-22 13:58:05', 'kp_rating': '9.1', 'year': '2019'}


StopIteration: ignored

3) Выгрузите весь Топ-500 и превратите его в список словарей. Отсортируйте этот список по убыванию рейтинга 








In [0]:
sorted_list_of_dicts = sorted(list_of_dicts, key=lambda f: f['kp_rating'], reverse=True) 
sorted_list_of_dicts

[{'_utc_timestamp': '2019-12-22 07:13:02',
  'kp_rating': '9.4',
  'name': 'FORD против FERRARI',
  'url': 'https://www.kinoafisha.info/movies/8354457/',
  'year': '2019'},
 {'_utc_timestamp': '2019-12-22 07:13:02',
  'kp_rating': '9.2',
  'name': 'Анастасия',
  'url': 'https://www.kinoafisha.info/movies/5640854/',
  'year': '1997'},
 {'_utc_timestamp': '2019-12-22 07:13:02',
  'kp_rating': '9.1',
  'name': 'Ржев',
  'url': 'https://www.kinoafisha.info/movies/8356720/',
  'year': '2019'},
 {'_utc_timestamp': '2019-12-22 07:13:02',
  'kp_rating': '9.1',
  'name': 'Холодное сердце 2',
  'url': 'https://www.kinoafisha.info/movies/8329619/',
  'year': '2019'},
 {'_utc_timestamp': '2019-12-22 07:13:02',
  'kp_rating': '9.1',
  'name': 'Малефисента: Владычица тьмы',
  'url': 'https://www.kinoafisha.info/movies/8326540/',
  'year': '2019'},
 {'_utc_timestamp': '2019-12-22 07:13:02',
  'kp_rating': '9.0',
  'name': 'Как приручить дракона 3',
  'url': 'https://www.kinoafisha.info/movies/8168520

4) Используйте один из контейнеров из модуля collections, чтобы собрать словарь вида "год выхода фильма -> количество фильмов из топ-500, вышедших в этот год". Выведите самый популярный год, а также минимальный и максимальный.

In [0]:
import collections
from collections import Counter

years = []

for f in list_of_dicts:
  years.append(f['year'])

c = Counter(years)
print(c)
print('Самый популярный год: ',c.most_common(1)[0][0])
print('Самый новый фильм вышел в {} году'.format(max(years)))
print('Самый старый фильм вышел в {} году'.format(min(years)))



Counter({2010: 36, 2018: 32, 2019: 29, 2011: 29, 2016: 25, 2009: 24, 2017: 22, 2012: 16, 2008: 14, 2013: 12, 2001: 11, 2006: 11, 2007: 11, 2014: 11, 1999: 8, 1995: 8, 2005: 8, 2000: 7, 2002: 7, 2015: 7, 1997: 6, 1994: 6, 1998: 6, 2004: 5, 1996: 5, 1984: 4, 2003: 3, 1990: 3, 1992: 3, 1968: 2, 1991: 2, 1986: 2, 1988: 2, 1983: 2, 1987: 2, 1980: 2, 1973: 1, 1969: 1, 1971: 1, 1977: 1, 1972: 1, 1981: 1, 1985: 1, 1976: 1, 1974: 1, 1962: 1, 1989: 1, 1957: 1, 1978: 1, 1975: 1, 1964: 1, 1982: 1, 1993: 1})
Самый популярный год:  2010
Самый новый фльм вышел в 2019 году
Самый старый фльм вышел в 1957 году


5) Напишите функцию, которая записывает ваш список словарей в файл в нужном формате, сортируя его по выбранному полю по возрастанию или убыванию.

In [0]:
from typing import List, Dict, Union, Any
import json
import csv
import jsonlines

def sort(order_by: Union[None, str] = None, # поле словарей, по которому будем сортировать
         ascending: bool = True):
  if ascending == True:
    ordered_list_of_dict  = sorted(list_of_dicts, key=lambda f: f[order_by], reverse=True)
  if ascending == False:
    ordered_list_of_dict  = sorted(list_of_dicts, key=lambda f: f[order_by], reverse=False)

  return ordered_list_of_dict
#Если сортировать по имени, сначала идут названия с цифрами в начале(типо '101 долматинец')


def dump(film_list: List[Dict[str, Any]], 
         filename: str, 
         filetype: str, 
         order_by: Union[None, str] = None, # поле словарей, по которому будем сортировать
         ascending: bool = True) -> None:
    
  filetypes = ['json', 'csv', 'tsv', 'jsonl']
  if filetype not in filetypes:
    raise ValueError('One of this filetypes is needed: json, csv, tsv or jsonl.')
  else:
    name = filename + '.' + filetype
    with open(name,'w',encoding='utf-8') as f:
      if filetype == 'json':
        for film in sort(order_by, ascending):
          json.dump(film, f, ensure_ascii=False)
      if filetype == 'csv':
        writer = csv.DictWriter(f, fieldnames = ['name', 'url', '_utc_timestamp','kp_rating', 'year'])
        writer.writeheader()
        for film in sort(order_by, ascending):
          writer.writerow(film)
      if filetype == 'tsv':
        writer = csv.DictWriter(f, fieldnames = ['name', 'url', '_utc_timestamp','kp_rating', 'year'], delimiter='\t')
        writer.writeheader()
        for film in sort(order_by, ascending):
          writer.writerow(film)
    if filetype == 'jsonl':
      with jsonlines.open(name, mode='w') as writer:
        for film in sort(order_by, ascending):
          writer.write(film)
        

      

dump(list_of_dicts, filename = 'myfile', filetype='json', order_by='name', ascending=False)
dump(list_of_dicts, filename = 'myfile', filetype='csv', order_by='name', ascending=False)
dump(list_of_dicts, filename = 'myfile', filetype='jsonl', order_by='name', ascending=False)
dump(list_of_dicts, filename = 'myfile', filetype='tsv', order_by='name', ascending=False)


Часть 2

1) Напишите функцию, возвращающую строку, представлящую случайный IPv4-адрес.

In [95]:
import random

def IPv4():
  ipv4 = ''
  for x in range(4):
    rr = random.randint(0,255)
    ipv4 += '.' + str(rr)
  return ipv4[1:]
  
IPv4()

'231.246.16.243'

2) Сгенерируйте список из 100 разных IPv4-адресов.

In [96]:
from collections import Counter

list_ipv4 = []
for x in range(100):
    list_ipv4.append(IPv4())

c = Counter(list_ipv4)
for k,v in c.items():
  if v > 1:
    list_ipv4.remove(k)
    list_ipv4.append(IPv4())

print(list_ipv4)

['114.199.81.98', '254.198.181.62', '15.244.1.15', '226.243.142.38', '235.157.236.146', '79.213.42.11', '97.133.12.120', '50.73.239.89', '163.148.90.112', '91.19.171.225', '7.127.201.248', '71.158.246.243', '122.60.204.160', '54.207.161.246', '158.121.249.14', '60.31.220.171', '82.190.4.36', '167.96.164.21', '156.63.70.247', '10.210.21.175', '135.255.186.86', '203.233.102.63', '113.0.139.73', '162.247.78.235', '246.138.11.127', '133.17.119.112', '214.83.196.139', '111.65.186.53', '82.186.161.197', '31.239.13.67', '247.59.95.8', '70.18.151.11', '63.254.69.31', '3.76.3.101', '4.94.6.191', '241.254.7.150', '14.173.249.238', '252.159.40.153', '26.209.38.217', '243.10.213.185', '239.152.127.212', '35.51.109.147', '124.159.28.199', '224.29.170.120', '83.238.131.199', '139.250.217.57', '59.182.210.109', '226.255.14.18', '221.103.80.210', '23.226.203.231', '166.226.119.24', '234.210.240.137', '80.207.217.246', '38.201.175.91', '195.219.155.222', '28.142.14.45', '139.247.151.224', '78.167.54.22

3) С помощью https://freegeoip.app/ выгрузите для каждого из этих адресов информацию в формате JSON. Делайте случайные перерывы от 0.5 до 1 секунды между запросами

In [97]:
import time
import requests

ips_jsn =[]

for ip in list_ipv4:
  url = "https://freegeoip.app/json/" + ip

  headers = {
      'accept': "application/json",
      'content-type': "application/json"
      }

  response = requests.request("GET", url, headers=headers)
  ip_jsn = json.loads(response.text)
  ips_jsn.append(ip_jsn)
  print(ip_jsn)
  time.sleep(random.uniform(0.5, 1))


{'ip': '114.199.81.98', 'country_code': 'ID', 'country_name': 'Indonesia', 'region_code': 'JK', 'region_name': 'Jakarta', 'city': 'Jakarta', 'zip_code': '', 'time_zone': 'Asia/Jakarta', 'latitude': -6.1741, 'longitude': 106.8296, 'metro_code': 0}
{'ip': '254.198.181.62', 'country_code': '', 'country_name': '', 'region_code': '', 'region_name': '', 'city': '', 'zip_code': '', 'time_zone': '', 'latitude': 0, 'longitude': 0, 'metro_code': 0}
{'ip': '15.244.1.15', 'country_code': 'US', 'country_name': 'United States', 'region_code': '', 'region_name': '', 'city': '', 'zip_code': '', 'time_zone': 'America/Chicago', 'latitude': 37.751, 'longitude': -97.822, 'metro_code': 0}
{'ip': '226.243.142.38', 'country_code': '', 'country_name': '', 'region_code': '', 'region_name': '', 'city': '', 'zip_code': '', 'time_zone': '', 'latitude': 0, 'longitude': 0, 'metro_code': 0}
{'ip': '235.157.236.146', 'country_code': '', 'country_name': '', 'region_code': '', 'region_name': '', 'city': '', 'zip_code':

4) В ответе этого API есть поле time_zone. Это часовой пояс местоположения, где теоретически находится сервер с таким IP.
Сгенерируйте случайный момент времени в 2019 году (по UTC времени).
Выведите его в виде unix timestamp
Приведите его ко всем встретившимся часовым поясам. Выведите результат.

In [129]:
import datetime

start = datetime.datetime(2019, 1, 1, 0, 0, 1)
end = datetime.datetime(2019, 12, 31, 23, 59, 59)
day_rand = start + abs(start - end)*random.random()
time = day_rand.replace(microsecond=0)
unix_time = int(time.timestamp())
print(time)
print(unix_time)

2019-11-08 21:33:09
1573248789


In [130]:
from datetime import datetime
import pytz

time_zones = []
for ip in ips_jsn:
  if ip['time_zone'] != '':
    if ip['time_zone'] not in time_zones:
      time_zones.append(ip['time_zone'])
time_zones

for time_zone in time_zones:
    needed_time = datetime.fromtimestamp(unix_time, pytz.timezone(time_zone))
    print(time_zone, ":", needed_time)


Asia/Jakarta : 2019-11-09 04:33:09+07:00
America/Chicago : 2019-11-08 15:33:09-06:00
Europe/Berlin : 2019-11-08 22:33:09+01:00
America/New_York : 2019-11-08 16:33:09-05:00
Asia/Tokyo : 2019-11-09 06:33:09+09:00
America/Los_Angeles : 2019-11-08 13:33:09-08:00
Pacific/Auckland : 2019-11-09 10:33:09+13:00
America/Sao_Paulo : 2019-11-08 19:33:09-02:00
Asia/Shanghai : 2019-11-09 05:33:09+08:00
Europe/Rome : 2019-11-08 22:33:09+01:00
Asia/Seoul : 2019-11-09 06:33:09+09:00
America/Detroit : 2019-11-08 16:33:09-05:00
Asia/Ho_Chi_Minh : 2019-11-09 04:33:09+07:00
Europe/Warsaw : 2019-11-08 22:33:09+01:00
Asia/Kolkata : 2019-11-09 03:03:09+05:30
America/Kentucky/Louisville : 2019-11-08 16:33:09-05:00
Europe/Vaduz : 2019-11-08 22:33:09+01:00
Europe/Istanbul : 2019-11-09 00:33:09+03:00
America/Fortaleza : 2019-11-08 18:33:09-03:00
Europe/Moscow : 2019-11-09 00:33:09+03:00
Europe/London : 2019-11-08 21:33:09+00:00
America/Panama : 2019-11-08 16:33:09-05:00
Africa/Johannesburg : 2019-11-08 23:33:09+0